In [28]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

dev = qml.device("default.qubit", wires=2)

def get_angles(x):

    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(
        np.sqrt(x[2] ** 2 + x[3] ** 2)
        / np.sqrt(x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + x[3] ** 2)
    )

    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])


def statepreparation(a):
    qml.RY(a[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)


def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])


@qml.qnode(dev, interface="autograd")
def circuit(weights, angles):
    for W in weights:
        layer(W)
    return qml.expval(qml.PauliZ(0))


data = np.loadtxt("iris_data.txt", delimiter = ',')
X = data[:, 0:2]

# pad the vectors to size 2^2 with constant values
padding = 0.3 * np.ones((len(X), 1))
X_pad = np.c_[np.c_[X, padding], np.zeros((len(X), 1))]

# normalize each input
normalization = np.sqrt(np.sum(X_pad ** 2, -1))
X_norm = (X_pad.T / normalization).T

# angles for state preparation are new features
features = np.array([get_angles(x) for x in X_norm], requires_grad=False)

np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
feats_train = features[index[:num_train]]


num_qubits = 2
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)

In [26]:
# Visualize feature mapping circuits
drawer = qml.draw(statepreparation)
print(drawer(feats_train[0]))

0: ──RY(0.09)─╭●────────────╭●──X────────╭●────────────╭●──X────────┤  
1: ───────────╰X──RY(-0.00)─╰X──RY(0.00)─╰X──RY(-0.45)─╰X──RY(0.45)─┤  


In [30]:
# Visualize the ansatz layers
drawer = qml.draw(circuit)
print(drawer(weights_init, feats_train[0]))

0: ──Rot(-0.01,0.01,0.00)─╭●──Rot(0.00,-0.00,0.00)──╭●──Rot(-0.01,-0.01,0.00)─╭●
1: ──Rot(0.01,0.00,0.01)──╰X──Rot(-0.00,0.02,-0.01)─╰X──Rot(0.00,0.00,0.02)───╰X

───Rot(-0.01,-0.01,-0.00)─╭●──Rot(-0.01,-0.01,0.01)─╭●──Rot(-0.01,-0.01,-0.02)─╭●─┤  <Z>
───Rot(0.02,0.02,-0.00)───╰X──Rot(0.00,0.01,-0.02)──╰X──Rot(0.02,-0.00,-0.01)──╰X─┤     
